In [21]:
import sqlite3
import pandas as pd

In [22]:
from imblearn.over_sampling import SMOTE

In [23]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve,f1_score, precision_score,recall_score,precision_recall_curve
import matplotlib.pyplot as plt
import joblib

In [24]:
from scipy.stats import uniform

In [25]:
from sklearn.preprocessing import StandardScaler
import pickle

In [26]:
conn = sqlite3.connect('fraud_detection.db')
#when only query, no need cursor.   cursor = conn.cursor()

df = pd.read_csv('creditcard.csv')
df.to_sql('transactions', conn, if_exists='replace', index=False)

query = """
SELECT * FROM transactions 
WHERE Amount > 0
"""
data = pd.read_sql_query(query, conn)

conn.close()

print(data.head())
# This code connects to a SQLite database, reads a CSV file into a DataFrame,
# and executes a SQL query to retrieve transactions with a positive amount.
print(data['Class'].value_counts())

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [27]:
print(data.isnull().sum()) #check for null values
data['Class'] = data['Class'].astype('int') #make sure Class is int

Q1 = data['Amount'].quantile(0.25)
Q3 = data['Amount'].quantile(0.75)
IQR = Q3 - Q1
outliners = data[(data['Amount'] < (Q1 - 1.5 * IQR)) | (data['Amount'] > (Q3 + 1.5 * IQR))]
print(f"Number of outliners: {len(outliners)}") # check for outliners in Amount column
# these outliners may be frauds, so we will keep them for now

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64
Number of outliners: 31661


In [28]:
# Convert Time to Hour and create Is_Night feature
data['Hour'] = data['Time'] // 3600 % 24  # Convert Time to Hour (0-23)
data['Is_Night'] = data['Hour'].apply(lambda x: 1 if 0<= x <= 6 else 0)

# Amount feature engineering, scaling and binning for better model performance

scaler = StandardScaler()

# standardize the Amount feature
data['Amount_Scaled'] = scaler.fit_transform(data[['Amount']])
data['Amount_high'] = data['Amount'].apply(lambda x: 1 if x > 300 else 0)
# save scaler.pkl
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("scaler.pkl saved！")

print(data.head(5))  # Display summary statistics of the DataFrame

scaler.pkl saved！
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V25       V26       V27       V28  Amount  \
0  0.098698  0.363787  ...  0.128539 -0.189115  0.133558 -0.021053  149.62   
1  0.085102 -0.255425  ...  0.167170  0.125895 -0.008983  0.014724    2.69   
2  0.247676 -1.514654  ... -0.327642 -0.139097 -0.055353 -0.059752  378.66   
3  0.377436 -1.387024  ...  0.647376 -0.221929  0.062723  0.061458  123.50   
4 -0.270533  0.817739  ... -0.206010  0.502292  0.219422  0.215153   69.99   

   Class  Hour  Is_Night  

In [29]:
# Resampling using SMOTE, to handle class imbalance, improve model performance,
# and ensure the model is not biased towards the majority class
# but may increase the risk of overfitting, need to compare with scale_pos_weight
x = data.drop(['Class', 'Time'], axis=1)
y = data['Class']
smote = SMOTE(random_state=42)
x_resampled, y_resampled = smote.fit_resample(x, y)
print(f"resampled class distribution:\n{y_resampled.value_counts()}")
x_resampled.to_csv('x_resampled.csv', index=False)
y_resampled.to_csv('y_resampled.csv', index=False)

resampled class distribution:
Class
0    282517
1    282517
Name: count, dtype: int64


In [30]:
# 1. load smote data
X = pd.read_csv('x_resampled.csv')
y = pd.read_csv('y_resampled.csv').values.ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("SMOTE训练集形状：", X_train.shape)
print("SMOTE测试集形状：", X_test.shape)

# 2. smote model training and evaluation
model_smote = XGBClassifier(random_state=42)  # default：n_estimators=100, max_depth=6, learning_rate=0.3
model_smote.fit(X_train, y_train)
y_pred_smote = model_smote.predict(X_test)
print("\nsolution1：SMOTE（default） - report：")
print(classification_report(y_test, y_pred_smote))
print(f"solution1 AUC-ROC: {roc_auc_score(y_test, model_smote.predict_proba(X_test)[:, 1])}")
# check if overfitting
print("SMOTE train F1：", f1_score(y_train, model_smote.predict(X_train)))
print("SMOTE test F1：", f1_score(y_test, y_pred_smote))



SMOTE训练集形状： (452027, 33)
SMOTE测试集形状： (113007, 33)

solution1：SMOTE（default） - report：
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56412
           1       1.00      1.00      1.00     56595

    accuracy                           1.00    113007
   macro avg       1.00      1.00      1.00    113007
weighted avg       1.00      1.00      1.00    113007

solution1 AUC-ROC: 0.9999944350080449
SMOTE train F1： 0.9999977868516858
SMOTE test F1： 0.9998321688205002


In [31]:
# 3. solution2 original data + scale_pos_weight
data = pd.read_csv('creditcard.csv')
data['Hour'] = (data['Time'] // 3600) % 24
data['Is_Night'] = data['Hour'].apply(lambda x: 1 if 0 <= x <= 6 else 0)
data['Amount_Scaled'] = StandardScaler().fit_transform(data[['Amount']])
data['Amount_high'] = data['Amount_Scaled'].apply(lambda x: 1 if x > 500 else 0)
X_orig = data.drop(['Class', 'Time'], axis=1)
y_orig = data['Class']

X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X_orig, y_orig, test_size=0.2, random_state=42)
fraud_ratio = len(y_train_orig[y_train_orig == 0]) / len(y_train_orig[y_train_orig == 1])
model_spw = XGBClassifier(scale_pos_weight=2*fraud_ratio, random_state=42)  # 增大权重
model_spw.fit(X_train_orig, y_train_orig)
y_pred_spw = model_spw.predict(X_test_orig)
print("\nsolution2：scale_pos_weight（default） - report：")
print(classification_report(y_test_orig, y_pred_spw))
print(f"solution2 AUC-ROC: {roc_auc_score(y_test_orig, model_spw.predict_proba(X_test_orig)[:, 1])}")
# check if overfitting
print("scale_pos_weight train F1：", f1_score(y_train_orig, model_spw.predict(X_train_orig)))
print("scale_pos_weight test F1：", f1_score(y_test_orig, y_pred_spw))


solution2：scale_pos_weight（default） - report：
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.99      0.83      0.90        98

    accuracy                           1.00     56962
   macro avg       0.99      0.91      0.95     56962
weighted avg       1.00      1.00      1.00     56962

solution2 AUC-ROC: 0.982696989882053
scale_pos_weight train F1： 1.0
scale_pos_weight test F1： 0.9


In [32]:
# 4. optimize （RandomizedSearchCV，optimize recall）
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': uniform(0.01, 0.3),  # use uniform distribution
}
random_search = RandomizedSearchCV(XGBClassifier(random_state=42), param_grid, n_iter=20, cv=5, scoring='recall', n_jobs=-1)
random_search.fit(X_train, y_train)
print("\nBEST prarmater（SMOTE，optimize recall）：", random_search.best_params_)
print("Best Recall score：", random_search.best_score_)
model_best = random_search.best_estimator_


BEST prarmater（SMOTE，optimize recall）： {'learning_rate': np.float64(0.2911852430188321), 'max_depth': 5, 'n_estimators': 150}
Best Recall score： 1.0


In [33]:
# evaluate model（modify threshold to 0.3）defaule threshhold 0.5
# y_pred = model_best.predict(X_test)   
y_pred_best = (model_best.predict_proba(X_test)[:, 1] > 0.3).astype(int)
print("\noptimized model（SMOTE，threshold 0.3） - report：")
print(classification_report(y_test, y_pred_best))
print(f"optimized AUC-ROC: {roc_auc_score(y_test, model_best.predict_proba(X_test)[:, 1])}")
# 检查过拟合
print("Optimized train F1：", f1_score(y_train, model_best.predict(X_train)))
print("Optimized test F1：", f1_score(y_test, y_pred_best))


optimized model（SMOTE，threshold 0.3） - report：
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56412
           1       1.00      1.00      1.00     56595

    accuracy                           1.00    113007
   macro avg       1.00      1.00      1.00    113007
weighted avg       1.00      1.00      1.00    113007

optimized AUC-ROC: 0.9999956117781676
Optimized train F1： 1.0
Optimized test F1： 0.9998056743101438


In [34]:
# 5. feature importance
importance = pd.Series(model_best.feature_importances_, index=X_train.columns).sort_values(ascending=False)
plt.figure(figsize=(10, 6))
importance[:10].plot(kind='bar')
plt.title('feature importance（XGBoost - SMOTE optimized）')
plt.xlabel('feature')
plt.ylabel('importance')
plt.savefig('feature_importance.png')
plt.close()
pd.DataFrame({'Feature': X_train.columns, 'Importance': model_best.feature_importances_}).to_csv('feature_importance.csv', index=False)
print("\nTop 10 importance：")
print(importance[:10])


Top 10 importance：
V14       0.626879
V4        0.054536
V12       0.038433
V8        0.024459
V1        0.023954
V10       0.020700
V17       0.018914
V3        0.017613
Amount    0.015633
V11       0.013697
dtype: float32


C:\Users\Liuji\AppData\Local\Temp\ipykernel_18488\3385652392.py:8: UserWarning: Glyph 65288 (\N{FULLWIDTH LEFT PARENTHESIS}) missing from font(s) DejaVu Sans.
  plt.savefig('feature_importance.png')
C:\Users\Liuji\AppData\Local\Temp\ipykernel_18488\3385652392.py:8: UserWarning: Glyph 65289 (\N{FULLWIDTH RIGHT PARENTHESIS}) missing from font(s) DejaVu Sans.
  plt.savefig('feature_importance.png')


In [35]:
# 6. model save（base recall）
final_model = model_best
#X_final, y_final, X_test_final, y_test_final = X, y, X_test, y_test
joblib.dump(final_model, 'fraud_model.pkl')

# 7. export（for Power BI）
# fpr, tpr, thresholds = roc_curve(y_test_final, final_model.predict_proba(X_test_final)[:, 1])
# roc_data = pd.DataFrame({'FPR': fpr, 'TPR': tpr, 'Thresholds': thresholds})
# roc_data.to_csv('roc_data.csv', index=False)
# results = X_test_final.copy()
# results['Actual'] = y_test_final
# results['Predicted'] = (final_model.predict_proba(X_test_final)[:, 1] > 0.3).astype(int)
# results['Fraud_Probability'] = final_model.predict_proba(X_test_final)[:, 1]
# results.to_csv('model_results.csv', index=False)

['fraud_model.pkl']